In [77]:
import pymupdf
import os
import pymupdf4llm
import getpass
from pydantic import BaseModel
import pandas as pd
import csv
import io

In [7]:
dic_path = '/Users/ashwanth/Data/Wines/May/Indents'

In [12]:
def get_all_files(dic_path):
    return os.listdir(dic_path)

In [13]:
files = get_all_files(dic_path)

In [35]:
def extract_data_from_file(file):
    doc = pymupdf.open(file)
    all_text = ""
    for page in doc:
        text = page.get_text()
        all_text += text
    return all_text

In [45]:
api_key = getpass.getpass("Enter API KEY: ")

Enter API KEY:  ········


In [56]:
import openai

client = OpenAI(
    api_key = api_key,
)


In [58]:
class InvoiceDataExtraction(BaseModel):
    data: pd.DataFrame

    class Config:
        arbitrary_types_allowed = True


In [122]:
all_data = []

for file in files:
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from the invoices and you return BrandName, Qty, Price in a csv format."},
        {"role": "user", "content": extract_data_from_file(os.path.join(dic_path,file))}
    ],
    #response_format=InvoiceDataExtraction,
    )
    csv_content = completion.choices[0].message.content.strip("```csv\n").strip("```")

    # Parse the CSV content
    csv_reader = csv.DictReader(io.StringIO(csv_content))
    data = pd.DataFrame(csv_reader, columns=['BrandName', 'Qty', 'Price'])
    all_data.append(data)
    

    

In [123]:
df = pd.concat(all_data, ignore_index=True, axis=0)

In [125]:
df.head()

,BrandName,Qty,Price
0,KING FISHER STRONG P,10,13010
1,KNOCKOUT HIGH PUNCH,30,39030
2,BUDWEISER KING OF BE,10,18010
3,BUDWEISER MAGNUM PRE,20,38020
4,KING FISHER PREMIUM,55,66055


In [126]:
from pandasai import Agent


python(31961) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [128]:
pandasai_api_key = getpass.getpass("Enter PandasAI API Key:")

Enter PandasAI API Key: ········


In [130]:
os.environ['PANDASAI_API_KEY'] = pandasai_api_key

In [131]:
agent = Agent(df)

In [133]:
agent.chat("what is the quantity of BrandName KING FISHER STRONG?")

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/prompt_generation.py", line 37, in execute
    self.logger.log(f"Using prompt: {prompt}")
                                    ^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/prompts/base.py", line 55, in __str__
    return self.to_string()
           ^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/prompts/base.py", line 50, in to_string
    self._resolved_prompt = self.prompt.render(**self.props)


"Unfortunately, I was not able to answer your question, because of the following error:\n\n'Index' object has no attribute '_format_native_types'\n"

In [134]:
from pandasai.llm.openai import OpenAI
llm = OpenAI(api_token=api_key)

pandas_ai = PandasAI(llm)
pandas_ai(df, prompt="what is the quantity of BrandName KING FISHER STRONG?")



NameError: name 'PandasAI' is not defined

In [135]:
import os
import pandas as pd
from pandasai import Agent

# Sample DataFrame
sales_by_country = pd.DataFrame({
    "country": ["United States", "United Kingdom", "France", "Germany", "Italy", "Spain", "Canada", "Australia", "Japan", "China"],
    "revenue": [5000, 3200, 2900, 4100, 2300, 2100, 2500, 2600, 4500, 7000]
})

# By default, unless you choose a different LLM, it will use BambooLLM.
# You can get your free API key signing up at https://pandabi.ai (you can also configure it in your .env file)
os.environ["PANDASAI_API_KEY"] = pandasai_api_key

agent = Agent(sales_by_country)
agent.chat('Which are the top 5 countries by sales?')

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/prompt_generation.py", line 37, in execute
    self.logger.log(f"Using prompt: {prompt}")
                                    ^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/prompts/base.py", line 55, in __str__
    return self.to_string()
           ^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/prompts/base.py", line 50, in to_string
    self._resolved_prompt = self.prompt.render(**self.props)


"Unfortunately, I was not able to answer your question, because of the following error:\n\n'Index' object has no attribute '_format_native_types'\n"